In [ ]:
# This example was taken from the langchain docs:
# https://python.langchain.com/v0.2/docs/tutorials/rag/
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai faiss-cpu

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./assets-resources/tree-of-thoughts.pdf")
documents = loader.load()

In [4]:
for doc in documents:
    print(doc.page_content[:400])

Tree of Thoughts: Deliberate Problem Solving
with Large Language Models
Shunyu Yao
Princeton UniversityDian Yu
Google DeepMindJeffrey Zhao
Google DeepMindIzhak Shafran
Google DeepMind
Thomas L. Grifﬁths
Princeton UniversityYuan Cao
Google DeepMindKarthik Narasimhan
Princeton University
Abstract
Language models are increasingly being deployed for general problem solving
across a wide range of tasks
GĮŔũƜ
jũƜŔũƜGĮŔũƜ
jũƜŔũƜʱÊʲˤGjʱæʲˤĵÉGĮŔũƜ
ˤjũƜŔũƜʱçʲˤĵÉˁʟʟʟʟaÊĠĵŗƓŤƆˤſĵŤòGĮŔũƜ
ˤjũƜŔũƜʱíʲˤÉĵÉˤʱĵũŗŝʲʟʟʟʟʟʟˤˤʝˤƛĎĵũĈĎƜ)L[FRORUE\<XTLDQ0DUNGLIIHUHQFHE\FRORU
GĮŔũƜ
jũƜŔũƜGĮŔũƜ
jũƜŔũƜGĮŔũƜ
ˤjũƜŔũƜʱçʲˤòĦƙˤĵĮŝƓŝŤòĮçƆˤƀƓƜĎˤĵÉˤʱĵÉˁʲaÊĠĵŗƓŤƆˤſĵŤòGĮŔũƜ
ˤjũƜŔũƜʱíʲˤÉŗòòˤĵƙˤĎĵũĈĎŤŝˤʱÉĵÉʲʟʟʟʟʟʟʟʟʟʟˤˤƛĎĵũĈĎƜ
ʱçʲˤĎÊđĮˤĵƙˤĎĵũĈĎƜˤŗĵĭŔƜđĮĈˤʱĵÉʲʱÊʲˤGĮŔũƜˁjũƜŔũƜˤŗĵĭŔƜđĮĈˤʱGjʲFigure 1: Schematic
Chain-of-thought (CoT) prompting [35] was proposed to address cases where the mapping of
inputxto outputyis non-trivial (e.g. when xis a math question and yis the ﬁnal numerical answer).
The key ide

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [7]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [8]:
retriever = db.as_retriever()

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "tree_of_thoughts",
    "Searches and returns excerpts from the tree of thoughts paper.",
)
tools = [tool]

In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

llm.invoke("Are you tired?")

AIMessage(content='I am an AI and do not experience physical fatigue like humans do. How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4c36f55b-a8a5-4c85-a480-edf8dde6b98e-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

In [11]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [12]:
result = agent_executor.invoke(
    {
        "input": "what is the tree of thoughts technique?"
    }
)

result["output"]

'The Tree of Thoughts technique is a framework for language model inference that allows models to perform deliberate decision-making by considering multiple reasoning paths and self-evaluating choices to decide the next course of action. It generalizes over the popular "Chain of Thought" approach and enables exploration over coherent units of text ("thoughts") that serve as intermediate steps toward problem-solving. This technique enhances language models\' problem-solving abilities on tasks that require exploration, strategic lookahead, or where initial decisions play a pivotal role.\n\nThe Tree of Thoughts framework involves decomposing the intermediate process into thought steps, generating potential thoughts from each state, heuristically evaluating states, and using a search algorithm to explore multiple reasoning paths over thoughts. It addresses the shortcomings of existing approaches by exploring different continuations within a thought process locally and incorporating plannin

In [13]:
result = agent_executor.invoke(
    {"input": "How did they evaluated the performance of the technique?"}
)

In [14]:
result["output"]

'The performance of the technique was evaluated using a framework called Tree of Thoughts (ToT). The ToT framework involves maintaining a tree of coherent language sequences that serve as intermediate steps toward problem-solving. The LM (Language Model) self-evaluates the progress of different intermediate thoughts through a deliberate reasoning process instantiated in language. This implementation of search heuristics via LM self-evaluation and deliberation is novel and allows for systematic exploration of the tree of thoughts with lookahead and backtracking.\n\nThe ToT framework was applied to three tasks: Game of 24, Creative Writing, and Crosswords. The tasks required deductive, mathematical, commonsense, and lexical reasoning abilities, as well as systematic planning or search. The ToT framework obtained superior results on all three tasks by being general and flexible enough to support different levels of thoughts, different ways to generate and evaluate thoughts, and different 